# Objetivos de cómputo en Azure ML

Son computadoras físicas o virtuales donde se ejecutan experimentos. Existen distintos tipos para adaptarse a tus necesidades.

1. Tipos de objetivos de cómputo
    - **Cómputo local**: Ideal para desarrollo y pruebas con poca data.
        Se ejecuta en el mismo dispositivo donde inicias el experimento (ej. nuestro pc, en la estacion de trabajo del notebook...).
    - **Clúster de cómputo**: Para alta escalabilidad con mucha data o procesamiento paralelo.
        Grupos de máquinas virtuales que se expanden o contraen según la demanda/necesidad.
    - **Cómputo adjunto**: Aprovecha entornos de cómputo ya existentes en Azure (ej. máquinas virtuales, Databricks).
        Útil para cargas de trabajo específicas.

2. Objetivos de cómputo para inferencia (solo para Azure ML Studio)
    - **Clústeres de inferencia**: Utilizan Azure Kubernetes Service para desplegar modelos entrenados como servicios de inferencia.

3. Beneficios de usar objetivos de cómputo
    - **Flexibilidad**: Desarrollar y probar en local, luego escalar a producción.
    - **Optimización de recursos**: Ejecutar procesos en el objetivo más adecuado (ej. CPU para entrenar, CPU solo para evaluar).
    - **Control de costes**: Pagar solo por el uso, iniciar y detener objetivos automáticamente, escalado automático.


#### 1. Creación de objetivos de computo

Las maneras más comunes de crear o asociar un compute target son, usar la página Compute en Azure ML Studio o usar el SDK.

##### - Creación de un objetivo de computo administrado con el SDK 
    
Es aquel por Azure ML, como un clúster de proceso de Azure Machine Learning.

Para crear un clúster de proceso de Azure Machine Learning, use la  clase `azureml.core.compute.ComputeTarget` y la  clase `AmlCompute`, como se muestra a continuación.

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, AmlCompute

# Load the workspace from the saved config file
ws = Workspace.from_config()

# Specify a name for the compute (unique within the workspace)
compute_name = 'aml-cluster'

# Define compute configuration
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',      # Creamos un objeto AmlCompute con la configuración necesaria   
                                                       min_nodes=0, max_nodes=4,        # Definimos el número mínimo y máximo de nodos
                                                       vm_priority='dedicated')         # Definimos la prioridad del clúster

# Create the compute
aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)                    # Creamos el clúster de cálculo
aml_cluster.wait_for_completion(show_output=True)                                       # Esperamos a que se complete la creación

##### - Adjuntar un objetivo de computo no gestionado con el SDK

Un compute target no gestionado es aquel que se define y gestiona fuera del espacio de trabajo de Azure ML; por ejemplo, una máquina virtual Azure o un clúster Azure Databricks.

El codigo usado para gestionar los compute targets no administrados es similar al de cómputo administrado, excepto que debe utilizar el método `ComputeTarget.attach()` para adjuntar el cómputo existente en función de sus ajustes de configuración específicos del objetivo.

Por ejemplo, este codigo se puede utilizar para adjuntar un clúster Azure Databricks existente

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import ComputeTarget, DatabricksCompute

ws = Workspace.from_config()

compute_name = 'db_cluster'

db_workspace_name = 'db_workspace'
db_resource_group = 'db_resource_group'
db_access_token = '1234-abc-5678-defg-90...'
db_config = DatabricksCompute.attach_configuration(resource_group=db_resource_group,        # Creamos un objeto DatabricksCompute con la configuración necesaria
                                                   workspace_name=db_workspace_name,        # Definimos el nombre del clúster de Databricks
                                                   access_token=db_access_token)            # Definimos el token de acceso

# Create the compute
databricks_compute = ComputeTarget.attach(ws, compute_name, db_config)                      # Creamos el clúster de Databricks
databricks_compute.wait_for_completion(True)

##### - Comprobación de la existencia de objetivos de cálculo

Si se quiere comprobar la existencia de un objetivo de cálculo y crear uno nuevo sólo si no hay ninguno con el nombre especificado. 
Para esto, podemos capturar la excepción `ComputeTargetException`

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_name = "aml-cluster"

# Check if the compute target exists
try:
    aml_cluster = ComputeTarget(workspace=ws, name=compute_name)
    print('Found existing cluster.')
except ComputeTargetException:
    # If not, create it
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
                                                           max_nodes=4)
    aml_cluster = ComputeTarget.create(ws, compute_name, compute_config)

aml_cluster.wait_for_completion(show_output=True)

#### 2. Uso de los objetivos de computo

Una vez se hayan creado o asociado los compute targets en la workstation, puede usarlos para ejecutar cargas de trabajo específicas; como son los experimentos.

Para usar un objetivo de computo determinado, puede especificarlo en el parámetro adecuado para una configuración de ejecución de experimentos. 

Por ejemplo, el siguiente codigo configura un estimador para usar el destino de proceso denominado aml-cluster

In [ ]:
from azureml.core import Environment, ScriptRunConfig

compute_name = 'aml-cluster'

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_dir',                      # Creamos un objeto ScriptRunConfig con la configuración necesaria
                                script='script.py',                             # Definimos el directorio y el script
                                environment=training_env,                       # Definimos el entorno
                                compute_target=compute_name)                    # Definimos el clúster de cálculo

Cuando se envía un experimento, la ejecución se pondrá en cola mientras se inicia el objetivo de computación aml-cluster y se crea en él el entorno especificado, y luego la ejecución se procesará en el entorno de computación.

En lugar de especificar el nombre del objetivo de computación, puede especificar un objeto `ComputeTarget`

In [ ]:
from azureml.core import Environment, ScriptRunConfig
from azureml.core.compute import ComputeTarget

compute_name = "aml-cluster"

training_cluster = ComputeTarget(workspace=ws, name=compute_name)

training_env = Environment.get(workspace=ws, name='training_environment')

script_config = ScriptRunConfig(source_directory='my_dir',                      # Creamos un objeto ScriptRunConfig con la configuración necesaria
                                script='script.py',                             # Definimos el directorio y el script
                                environment=training_env,                       # Definimos el entorno
                                compute_target=training_cluster)                # Definimos el clúster de cálculo con compute_target